In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed
from Surfer.ScriptRunner import ScriptRunner

In [5]:

def read_file_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'rb') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            
            binary_string = ''.join(format(byte, '08b') for byte in chunk)
            yield binary_string

# Example usage
file_path = 'rick___roll.jpg'
data = ''
for chunk in read_file_in_chunks(file_path):
    data += chunk

# len(acc) - 796*8

def write_binary_string_to_file(binary_string, output_file_path):
    with open(output_file_path, 'wb') as file:
        for i in range(0, len(binary_string), 8):
            byte = binary_string[i:i + 8]
            if len(byte) < 8:
                byte = byte + '0' * (8 - len(byte))
            file.write(bytes([int(byte, 2)]))

# Example usage
binary_string = data
output_file_path = 'out/output_file.jpg'
write_binary_string_to_file(binary_string, output_file_path)


In [6]:
import Stegasus 

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros

('hi, how atre you?', '01001011111')

Setting up Typoceros Completed



In [7]:
import importlib
importlib.reload(Stegasus)

<module 'Stegasus' from 'c:\\Users\\nasoo\\Desktop\\Y5Project\\Stegasus\\__init__.py'>

In [8]:
from Bot import *
from Chatter import prompts

In [9]:
driver = GetBrowser()

In [10]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [11]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)

In [12]:
run_script = ScriptRunner(driver,'./Surfer/scripts/')

In [13]:
class RedditPostFeed(Feed):
	def __init__(self, driver:webdriver.Chrome ,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Post Feed',driver=driver,handle=handle)
		# Store the WebDriver instance
		self.verbose = verbose
		self.focus()
		self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.catch first comment',self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				self.run_script('reddit.post.catch first comment',verbose=self.verbose)()
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.post.parse comment',verbose=self.verbose)()
			self.run_script('reddit.post.cursor next')()

class RedditFeed(Feed):
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Feed',handle=handle,driver=driver)
		self.verbose = verbose
		if not skip:
			self.driver.get('https://www.reddit.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.init_reddit',verbose=self.verbose)()
	

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.parse post',verbose=self.verbose)()
			self.run_script('surfer.cursor next')()

	def click_post(self, newTab=True):
		for i in range(10): # 10 retries
			self.focus()
			if i == 9:
				raise Exception('cursor not present')
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			else:
				break
  
		id = uuid()
		self.run_script('surfer.mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		h3 = element.find_element(By.CSS_SELECTOR,'h3')
		element = element if h3 is None else h3
		actions = ActionChains(driver)
		actions \
			.click(element) \
			.perform()
			# .key_down(Keys.CONTROL) \
			# .key_up(Keys.CONTROL) \
		return RedditPostFeed(self.driver,self.handle,verbose=self.verbose)

	def close(self):
		self.focus()
		# Close the WebDriver instance
		self.driver.quit()


In [14]:
# rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [15]:
# for x in islice(rf,15):
#   print(x)
#   time.sleep(2*max(1,len(x) * 0.01))

In [16]:
# e = rf.click_post()

In [17]:
# run_script('reddit.post.parse comment')()

In [18]:
from typing import Tuple


class ChatGPTWebsite:
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		self.name = 'GPT'
		self.handle = handle
		self.driver = driver
		self.verbose = verbose
		self.run_script = ScriptRunner(driver,'./Surfer/scripts/')
		if not skip:
			self.driver.get('https://chat.openai.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
	def getLandingNewChat(self):
		return ChatGPT(self.driver)

class ChatGPT:
	def __init__(self, driver:webdriver.Chrome, name:str|None = None):
		self.driver = driver
		self.run_script = ScriptRunner(driver,'./Surfer/scripts/')
		self.name = name

	def ask(self,prompt:str, newChat:bool = False) -> Tuple[str, 'ChatGPT']:
		if newChat:
			self.run_script('chat.openai.new chat')()
			WebDriverWait(self.driver, 30).until(
				EC.presence_of_element_located((By.CSS_SELECTOR, 'svg > circle+line+line+line+line+line+line+line+line'))# examples sun in new chat
			)
		else:
			if self.name:
				self.run_script('chat.openai.switch to chat name')(self.name)
				time.sleep(4)
		textarea = self.driver.find_element(By.CSS_SELECTOR,'textarea')

		JS_ADD_TEXT_TO_INPUT = """
			var elm = document.querySelector('textarea'), txt = arguments[0];
			elm.value += txt;
			elm.dispatchEvent(new Event('change'));
			"""
		prompts = prompt.split('\n')
		for line in prompts:
			self.driver.execute_script(JS_ADD_TEXT_TO_INPUT, line)
			textarea.send_keys(Keys.SHIFT + Keys.ENTER)
		time.sleep(2)
		self.driver.find_element(By.CSS_SELECTOR,'textarea + button').click()
		svg_path = 'M20.49 9A9 9 0 0 0 5.64 5.64L1 10m22 4l-4.64 4.36A9 9 0 0 1 3.51 15'
		WebDriverWait(self.driver, 60).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, f'[d="{svg_path}"]'))
		)
		time.sleep(5)
		if self.name is None: # first ChatGPT instance ever or new is pressed and we are already at new
			self.name = self.run_script('chat.openai.get current chat name')()
			return self.run_script('chat.openai.get last answer')(), self
		elif newChat:
			name = self.run_script('chat.openai.get current chat name')()
			return self.run_script('chat.openai.get last answer')(), ChatGPT(self.driver,name)
		else:
			return self.run_script('chat.openai.get last answer')(), self

In [19]:
gptw = ChatGPTWebsite(driver,skip=False)

In [20]:
alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')

In [21]:
actualChat = []

In [22]:
data

'111111111101100011111111111000000000000000010000010010100100011001001001010001100000000000000001000000010000000100000000011000000000000001100000000000000000000011111111110110110000000001000011000000000000011000000100000001010000011000000101000001000000011000000110000001010000011000000111000001110000011000001000000010100001000000001010000010100000100100001001000010100001010000001110000011110000110000010000000101110001010000011000000110000001011100010100000101100001011000011010000111010010010100011111000110100001101100100011000111000001011000010110001000000010110000100000001000110010011000100111001010010010101000101001000110010001111100101101001100000010110100101000001100000010010100101000001010010010100011111111110110110000000001000011000000010000011100000111000001110000101000001000000010100001001100001010000010100001001100101000000110100001011000011010001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100000101000001010000010100

In [23]:
demo_post = '''Hi there! recently unstaked IMX from the staking L2 IMX plattform. And withdrawed from L2 to L1 on the Immutable marketplace, as indicated. It says there the withdrawal process can take up to 48 hours. BUT it is taking much more than that. 4 days and still nothing. Any hints?

thanks for any info related guys'''

In [24]:
actualChat

[]

In [25]:
time.sleep(6)

In [27]:
# aliceChat = gptw.getLandingNewChat()
# a,_ = aliceChat.ask(prompts.start_conversation_with_post(alice,bob,demo_post))
# # a = actualChat[0]
# print(1,a)
# actualChat.append(a)
# a, rem = Stegasus.StegasusEncode(a,data)
# print(1,a)
# a,bobChat = aliceChat.ask(prompts.startChatReplyPrompt(Message(alice,a)),True)
# print(2,a)
# actualChat.append(a)
# a, rem = Stegasus.StegasusEncode(a,rem)
# print(2,a)
# a,_ = aliceChat.ask(prompts.startChatReplyPrompt(Message(bob,a)))
# print(3,a)
# actualChat.append(a)
# a, rem = Stegasus.StegasusEncode(a,rem)
# print(3,a)
a = """Thanks for the advice! I'll definitely reach out to Immutable Support for help. How's your day been otherwise?"""
for i in range(4,12):
	a,_ = bobChat.ask(prompts.inChatReplyPrompt(Message(alice,a)))
	print(i,a)
	actualChat.append(a)
	a, rem = Stegasus.StegasusEncode(a,rem)
	print(i,a)
	# switch
	a,_ = aliceChat.ask(prompts.inChatReplyPrompt(Message(bob,a)))
	print(i,a)
	actualChat.append(a)
	a, rem = Stegasus.StegasusEncode(a,rem)
	print(i,a)

4 None


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
len(rem)

2233

In [ ]:
len('010111010101101000100110'*100) - len(rem)

167

In [ ]:
167 / 8

20.875